In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mingptf.model import GPT
import numpy as np
import tensorflow as tf
from mingptf.bpe import BPETokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

2023-01-09 20:59:52.544788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/abhaykumar/opt/anaconda3/envs/mingpt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'

In [4]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning


2023-01-09 21:00:00.920091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of parameters: 124.44M


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tfgpt2lm_head_model/transformer/wpe/embeddings:0 ---- gpt/wpe/embeddings:0
tfgpt2lm_head_model/transformer/wte/weight:0 ---- gpt/wte/embeddings:0
tfgpt2lm_head_model/transformer/h_._0/ln_1/gamma:0 ---- gpt/block/ln_1/gamma:0
tfgpt2lm_head_model/transformer/h_._0/ln_1/beta:0 ---- gpt/block/ln_1/beta:0
tfgpt2lm_head_model/transformer/h_._0/attn/c_attn/weight:0 ---- gpt/block/causal_self_attention/c_attn/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/attn/c_proj/weight:0 ---- gpt/block/causal_self_attention/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/ln_2/gamma:0 ---- gpt/block/ln_2/gamma:0
tfgpt2lm_head_model/transformer/h_._0/ln_2/beta:0 ---- gpt/block/ln_2/beta:0
tfgpt2lm_head_model/transformer/h_._0/mlp/c_fc/weight:0 ---- gpt/block/mlp/c_fc/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/mlp/c_proj/weight:0 ---- gpt/block/mlp/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._1/ln_1/gamma:0 ---- gpt/block_1/ln_1/gamma:0
tfgpt2lm_head_model/transformer/h_._1/l

0.0
tfgpt2lm_head_model/transformer/ln_f/gamma:0 ---- gpt/ln_f/gamma:0
tfgpt2lm_head_model/transformer/ln_f/beta:0 ---- gpt/ln_f/beta:0


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

In [6]:
tokenizer.tokenize("hi")

['hi']

In [24]:
tokenizer = BPETokenizer()
hf_tokenizer = GPT2Tokenizer.from_pretrained(model_type)

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = [[tokenizer.encoder.encoder['<|endoftext|>']]]
        else:
            x = tokenizer(prompt)
    else:
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = hf_tokenizer(prompt, return_tensors='tf').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, top_k=40)
    
    out = tokenizer.decode(y.numpy())
    print(out)
        

In [25]:
generate(prompt='Andrej Karpathy, the', steps=20)

Andrej Karpathy, the chief executive officer of Karpathy Ventures, told Reuters."As we started looking at the markets we
